# PyCUDA primal-dual algorithms for TV/TGV-constrained imaging problems - Example notebook
This notebook's purpose is to reproduce the numerical experiments in the article:
> Kristian Bredies. Recovering piecewise smooth multichannel images by minimization of convex functionals with total generalized variation penalty. _Lecture Notes in Computer Science_, 8293:44-77, 2014.

In particular, it demonstrates the implemented primal-dual algorithms for solving variational problems with total variation (TV) as well as total generalized variation (TGV) regularization, i.e., problems of the type
$$ \min_u \ D(u) + R(u) $$
where $D$ is a discrepancy functional associated to an imaging problem and $R$ is either the (discrete) total variation
$$
R(u) = \alpha_1 \mathrm{TV}(u) = \alpha_1 \| \nabla u \|_{1}
$$
for $\alpha_0 > 0$ or the (discrete) total generalized variation of second order
$$
R(u) = \mathrm{TGV}_\alpha^2(u) = \min_w \ \alpha_1 \| \nabla u - w \|_1 + \alpha_0 \| \mathcal{E} w \|_1
$$
for $\alpha_0 > 0$, $\alpha_1 > 0$ and $\mathcal{E}w = \frac12(\nabla u + \nabla u^\perp)$ the symmetrized derivative of the vector field $w$. The computations are performed on regular rectangular grids in two dimensions and with multichannel data such as color images. Please confer the above-referenced publication for more details.

## Initialization
First, we import the necessary modules.

In [ ]:
import pycuda.autoinit
from matplotlib.pyplot import subplots, imread, imshow

## Denoising examples
The following example code calls the implemented primal-dual algorithm for image denoising with TV/TGV regularization. This method aims at solving the optimization problem
$$
\min_u \ \frac1p \| u - f \|_p^p + R(u)
$$
for $f$ a noisy image and $p \in \{1, 2\}$. The corresponding functions are `tv_denoise` and `tgv_denoise` from `denoise_pycuda`. They are called as follows:
```
tv_denoise(f, alpha1, norm, maxiter, vis)
tgv_denoise(f, alpha1, fac, norm, maxiter, vis)
```
Here, `f` is the noisy image, `alpha1` corresponds to the parameter $\alpha_1$, `fac` determines $\alpha_0 = \alpha_1 \cdot \mathrm{fac}$, `norm` corresponds to $p$, `maxiter` is the number of iterations and `vis` triggers a visualization of the current iterate every `vis`th iteration if positive.

In [ ]:
from denoise_pycuda import tv_denoise, tgv_denoise
from numpy import random, clip

### Example 1: $L^2$-denoising
This example demonstrates the case $p=2$. Gaussian noise with variance $\sigma = 0.2$ is added and the denoising methods are called with $\alpha_1$ that give best PSNR values within a reasonable number of iterations.

In [ ]:
random.seed(14031621)
u_clean = imread("test_data/alinas_eye.png")
f = u_clean + random.randn(*u_clean.shape) * 0.2
u_tv = tv_denoise(f, 0.305, 2, 500, -1)
u_tgv = tgv_denoise(f, 0.288, 2.0, 2, 500, -1)

The results can be visualized via `matplotlib.pyplot`.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(13, 10))
ax1.imshow(u_clean); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(f, 0, 1)); ax2.axis('off'); ax2.set_title('noisy image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV denoising')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV denoising')

### Example 2: $L^2$-denoising
This example reproduces Fig. 2 in *LNCS 8293:44-77, 2014*. Again, $p=2$ and $\alpha_1$ such that the results are PSNR-optimal.

In [ ]:
# computation
random.seed(14031621)
u_clean = imread("test_data/wet_leaf.png")
f = u_clean + random.randn(*u_clean.shape) * 0.2
u_tv = tv_denoise(f, 0.31, 2, 500, -1)
u_tgv = tgv_denoise(f, 0.298, 2.0, 2, 500, -1)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(13, 10))
ax1.imshow(u_clean); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(f, 0, 1)); ax2.axis('off'); ax2.set_title('noisy image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV denoising')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV denoising')

### Example 3: $L^1$-denoising

This example demonstrates the case $p=1$ and reproduces Fig. 3 in *LNCS 8293:44-77, 2014*. Impulsive noise is applied (1/3 of the data is replaced by random values) and the denoising methods are called with PSNR-optimal $\alpha_1$.

In [ ]:
# computation
random.seed(14031621)
u_clean = imread("test_data/balloons2.png")
noise = random.rand(*u_clean.shape)
pattern = random.rand(*u_clean.shape) < 1/3
f = u_clean * (1 - pattern) + noise * pattern
u_tv = tv_denoise(f, 0.86, 1, 1000, -1)
u_tgv = tgv_denoise(f, 0.82, 2.0, 1, 1000, -1)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(10, 10))
ax1.imshow(u_clean); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(f, 0, 1)); ax2.axis('off'); ax2.set_title('noisy image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV denoising')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV denoising')